In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Mengunakan Model**

In [2]:
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [36]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Load tokenizer dan model yang sudah dilatih
model_path = "/content/drive/MyDrive/bert_intent_model"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Inisialisasi pipeline untuk klasifikasi teks
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


Device set to use cpu


In [17]:
test_texts = [
    "aku sekarang sedang di pukul",
]

predictions = classifier(test_texts)

for text, pred in zip(test_texts, predictions):
    label_index = int(pred["label"].split("_")[1])
    intent = reverse_labels[label_index]
    print(f"Teks: {text}\nPrediksi Intent: {intent}\n")

Teks: aku sekarang sedang di pukul
Prediksi Intent: laporan-kekerasan



**model t5**

In [42]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = "/content/drive/MyDrive/t5-model/checkpoint-2000"

# Load model dan tokenizer dari checkpoint
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)


In [46]:
# Input (misalnya formatnya sama seperti training: pakai prefix "conversation:")
input_text = "saya ingin pesan tiket pesawat"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate output
output_ids = model.generate(input_ids, max_length=50)
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Jawaban model:", output)

Jawaban model: Saya ingin ingin tiket pesawat dan ingin tiket pesawat dan ingin tiket pesawat dan ingin ingin tik


In [91]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# 1. Load model dan tokenizer dari checkpoint kamu
model_path = "/content/drive/MyDrive/t5-model/checkpoint-2000"
t5_model = T5ForConditionalGeneration.from_pretrained(model_path)
t5_tokenizer = T5Tokenizer.from_pretrained(model_path)

# 2. Baca file CSV
csv_path = "/content/informasi_kdrt.csv"
df = pd.read_csv(csv_path)

# 3. Gabungkan semua isi CSV jadi teks panjang (knowledge base)
knowledge_base = ""
for index, row in df.iterrows():
    knowledge_base += (
        f"Jenis: {row['Jenis KDRT']}. "
        f"Penjelasan: {row['Penjelasan']}. "
        f"Dampak: {row['Dampak']}. "
        f"Contoh: {row['Contoh']}. "
        f"Sumber: {row['Sumber Bantuan']}. "
    )

# 4. Buat pertanyaan
pertanyaan = "What are examples of physical domestic violence?"

# 5. Gabungkan knowledge base + pertanyaan jadi satu input ke model
input_text = f"{knowledge_base} Pertanyaan: {pertanyaan}"

# 6. Tokenisasi dan batasi panjang input agar tidak melebihi batas model
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 7. Generate jawaban dari model
output_ids = t5_model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

# 8. Decode jawaban
jawaban = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 9. Tampilkan jawaban
print("Jawaban:", jawaban)


Jawaban: Penjelasan: Is there a way to get out of the house? Dampak: Is there a way out?


# **Mengabungkan**

In [28]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [37]:
model_path = "/content/drive/MyDrive/t5-model/checkpoint-2000"

# Load model dan tokenizer dari checkpoint
t5_model = T5ForConditionalGeneration.from_pretrained(model_path)
t5_tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load tokenizer dan model yang sudah dilatih
model_path = "/content/drive/MyDrive/bert_intent_model"
bert_tokenizer  = BertTokenizer.from_pretrained(model_path)
bert_tokenizer  = BertForSequenceClassification.from_pretrained(model_path)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


Device set to use cpu


In [52]:
# Mapping label ke intent
reverse_labels = {
    0: "kata_kasar",
    1: "laporan_kekerasan",
    2: "psikologi",
    3: "lainnya"
}


In [53]:
# Fungsi prediksi intent baru
def prediksi_intent(text):
    result = classifier(text)[0]  # Ambil hasil pertama
    label_index = int(result["label"].split("_")[1])  # Misal "LABEL_2" → ambil 2
    return reverse_labels[label_index]

In [57]:
def generate_response(text):
    input_text = f"{text.strip().lower()}. tolong bantu saya."
    input_ids = t5_tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = t5_model.generate(
        input_ids,
        max_length=50,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    output = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output


In [55]:
prediksi_intent("saya sedang depresi")

'psikologi'

In [50]:
generate_response("hello")

"Hello, I'm a newbie and I'm looking for a new job."